## Dependencies

In [77]:
import datetime
import pandas as pd
import geopandas as gpd
from dataclasses import dataclass
import uuid

from shapely.geometry import MultiPolygon, Polygon

from farsiteutils import User, Run_File, Config_File, Input, FilePaths, Database

from multiprocessing import Pool

from ipyleaflet import Map, basemaps, basemap_to_tiles, ScaleControl, LayersControl, WKTLayer, WidgetControl
from ipyleaflet.leaflet import LayerException
from ipywidgets import Layout, FloatProgress, HBox, VBox, FloatText
from ipywidgets import SelectionSlider

from functools import partial

import os

In [3]:
fp = FilePaths('/home/tcaglar/data/')
usr = User(fp)

Setting up the interface
Database interaction not yet implemented. Use pickle file for dataframes instead!


In [4]:
usr.UI

{'windspeedlow': 0, 'windspeedhigh': 20, 'windspeeddelta': 5, 'winddirectionlow': 0, 'winddirectionhigh': 180, 'winddirectiondelta': 20, 'relhumid': 10, 'burntime': datetime.timedelta(0, 7200), 'burntimestep': datetime.timedelta(0, 1800), 'temperature': 60}
Choosing a perimeter from the database


In [5]:
usr.mainapi.run_farsite()

DriverError: /home/tcaglar/data/20220330/Run_0034/out_Perimeters.shp: No such file or directory

In [148]:
def initiate_wktlayers(gdf, newws, newwd):
    add_mask = ((gdf['windspeed'] == newws) & 
                   (gdf['winddirection'] == newwd))
    
    gdf.loc[add_mask, 'WKTLayer'].apply(lambda wlayer: m.add_layer(wlayer))
    
def update_wktlayers(gdf, oldws, newws, oldwd, newwd):    
    remove_mask = ((gdf['windspeed'] == oldws) & 
                   (gdf['winddirection'] == oldwd))
    add_mask = ((gdf['windspeed'] == newws) & 
                   (gdf['winddirection'] == newwd))
    
    gdf.loc[remove_mask, 'WKTLayer'].apply(lambda wlayer: m.remove_layer(wlayer))
    gdf.loc[add_mask, 'WKTLayer'].apply(lambda wlayer: m.add_layer(wlayer))
    
def test_observe(m, gdf, vbox, event):
    if event['owner'].description == 'Wind Speed':
        oldws = event['old']
        newws = event['new']
        oldwd = vbox.children[1].value
        newwd = vbox.children[1].value
    elif event['owner'].description == 'Wind Direction':
        oldws = vbox.children[0].value
        newws = vbox.children[0].value
        oldwd = event['old']
        newwd = event['new']
        
    update_wktlayers(gdf, oldws, newws, oldwd, newwd)

In [179]:
gdf = usr.mainapi.db.gdfsimulation.to_crs(epsg=4326)
gdf['WKTLayer'] = gdf.apply(lambda row: WKTLayer(wkt_string = row['geometry'].wkt), axis=1)
# gdf['WKTLayer_visibility'] = False

centerlon = gdf.iloc[0]['geometry'].centroid.x
centerlat = gdf.iloc[0]['geometry'].centroid.y

m = Map(
    basemap=basemaps.Esri.WorldTopoMap,
    center=(centerlat, centerlon),
    zoom=10,
    layout=Layout(height='700px')
)

m.add_control(ScaleControl())
    
ws = gdf['windspeed'].unique()
wd = gdf['winddirection'].unique()
dt = gdf['datetime'].unique()

ws_select = SelectionSlider(description='Wind Speed', options=ws)
wd_select = SelectionSlider(description='Wind Direction', options=wd)

vbox = VBox([ws_select, wd_select, dt_select])

ws_select.observe(partial(test_observe, m, gdf, vbox), names='value')
wd_select.observe(partial(test_observe, m, gdf, vbox), names='value')

widget_control = WidgetControl(widget = vbox)
m.add_control(widget_control)    
    
initiate_wktlayers(gdf, vbox.children[0].value, vbox.children[1].value)

m

[34.31457332063798, -119.04471211379085]

In [180]:
m

Map(center=[34.31457332063798, -119.04471211379085], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [184]:
import requests
import arrow

baseURL = "https://firemap.sdsc.edu/pylaski/stations/data/latest?"

## Always start with the base of the URL

selectionType="selection=closestTo"
#15.87, -97.08
lat = 34.31457332063798 
lon = -119.04471211379085

## Latitude and longitude of the White House according to Google
selectionParameters = "&lat=%s&lon=%s" % (str(lat),str(lon))
observables = "&observable=temperature"

to = arrow.Arrow.now()
frm = to.shift(days=-1)
urlDateTime = "&from=%s&to=%s" % ( str(frm) , str(to) )

urlPlot = baseURL + selectionType + selectionParameters + observables + urlDateTime
print(urlPlot)

r = requests.get(urlPlot)
## Request to GET information from the given URL (Our REST query we built)
r_json = r.json()
# ## Extract the JSON object from the data returned on our GET request

# rTemperature = r_json['features'][0]['properties']['temperature']
# rTime = r_json['features'][0]['properties']['timestamp']

# rTimeMins = []
# for i, val in enumerate(rTime):
# 	rTimeMins.append( (arrow.get(rTime[i]).timestamp - arrow.get(rTime[0]).timestamp) / 60 / 60 )

# plt.plot(rTimeMins,rTemperature,'-og',label='Actual')
# plt.xlabel("time (hours)")
# plt.ylabel("temperature (F)")
# minFive = int(min(rTemperature)) - (int(min(rTemperature))%5)
# maxFive = (int(max(rTemperature)) +5) - ( (int(max(rTemperature)) +5) % 5 )
# plt.yticks(np.arange(minFive, maxFive+1, (maxFive - minFive)/2))
# plt.gca().yaxis.grid(True)
# title = "Temperature at Alert Airport, NU\n(Past 24 Hours)"
# plt.title(title)
# plt.show()

https://firemap.sdsc.edu/pylaski/stations/data/latest?selection=closestTo&lat=34.31457332063798&lon=-119.04471211379085&observable=temperature&from=2022-03-29T15:23:43.517905-07:00&to=2022-03-30T15:23:43.517905-07:00


In [185]:
r_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-119.04174, 34.29971]},
   'properties': {'description': {'id': '90558',
     'name': 'SCE Hondo Barranca Rd',
     'provider': 'Mesowest',
     'wifire_uid': 'mesowest_se209'},
    'distanceFromLocation': {'value': 1.632131030311548, 'units': 'km'},
    'temperature': {'timestamp': '2022-03-30 15:10:00-0700',
     'value': 61.65,
     'units': 'F'},
    'relative_humidity': {'timestamp': '2022-03-30 15:10:00-0700',
     'value': 76.23,
     'units': '%'},
    'wind_speed': {'timestamp': '2022-03-30 15:10:00-0700',
     'value': 2.36,
     'units': 'mps'},
    'wind_gust': {'timestamp': '2022-03-30 15:10:00-0700',
     'value': 5.92,
     'units': 'mps'},
    'wind_direction': {'timestamp': '2022-03-30 15:10:00-0700',
     'value': 186.8,
     'units': 'd'}}}]}

### Create Landscape file

In [171]:
gdf_5070 = usr.mainapi.db.gdfignition.set_crs(epsg=5070)
gdf_4326 = gdf_5070.to_crs(epsg=4326)
geom_5070 = gdf_5070.iloc[0]['geometry']
geom = gdf_4326.iloc[0]['geometry']

In [207]:
%%time

bounds = geom_5070.bounds

ulx = bounds[0]-10000
uly = bounds[3]+10000
lrx = bounds[2]+10000
lry=  bounds[1]-10000

fname_lst = {'density': 'US_140CBD_12052016/Grid/us_140cbd', 
             'base': 'US_140CBH_12052016/Grid/us_140cbh', 
             'cover': 'US_140CC_12052016/Grid/us_140cc', 
             'height': 'US_140CH_12052016/Grid/us_140ch', 
             'fuel': 'US_140FBFM40_20180618/Grid/us_140fbfm40', 
             'aspect': 'Aspect/Grid/us_asp', 
             'elevation': 'DEM_Elevation/Grid/us_dem', 
             'slope': 'Slope/Grid/us_slp'}
type_lst = {'density': 'cbd',
            'base': 'cbh',
            'cover': 'cc',
            'height': 'ch',
            'fuel': 'fuel',   # fbfm40
            'aspect': 'aspect',
            'elevation': 'elevation', #dem
            'slope': 'slope'}

from_folder = os.path.join('/data', 'firemap', 'landfire', 'mosaic')
to_folder = '/home/tcaglar/farsite/inputs/landscapes/'

# Create the asc files
ascpath_lst = {}
for (key, fname) in fname_lst.items():
    ascpath_lst[key] = f'{os.path.join(to_folder, "test")}-{key}.asc'
    os.system(f'gdal_translate -of AAIGrid -a_nodata -32768 -projwin {ulx} {uly} {lrx} {lry} {os.path.join(from_folder, fname)} {ascpath_lst[key]}')
    
# # Extra for the tif file
# os.system(f'gdal_translate -of GTiff -a_nodata -32768 -projwin {ulx} {uly} {lrx} {lry} {os.path.join(from_folder, fname_lst["elevation"])} {os.path.join(to_folder, "test")}-elevation.tif')


CPU times: user 0 ns, sys: 272 ms, total: 272 ms
Wall time: 2.76 s


In [208]:
lcppath = f'{os.path.join(to_folder, "test")}-maria'
# ascpath_lst

In [209]:
lcpmakepath = '/home/tcaglar/farsite/src/lcpmake'

base_command = f'{lcpmakepath} -latitude {m.center[0]} -landscape {lcppath}'
run_command = base_command
for (key, ascpath) in ascpath_lst.items():
    run_command += f' -{key} {ascpath}'
    
os.system(run_command)

0

In [200]:
ascpath_lst

{'density': '/home/tcaglar/farsite/inputs/landscapes/test-density.asc',
 'base': '/home/tcaglar/farsite/inputs/landscapes/test-base.asc',
 'cover': '/home/tcaglar/farsite/inputs/landscapes/test-cover.asc',
 'height': '/home/tcaglar/farsite/inputs/landscapes/test-height.asc',
 'fuel': '/home/tcaglar/farsite/inputs/landscapes/test-fuel.asc',
 'aspect': '/home/tcaglar/farsite/inputs/landscapes/test-aspect.asc',
 'elevation': '/home/tcaglar/farsite/inputs/landscapes/test-elevation.asc',
 'slope': '/home/tcaglar/farsite/inputs/landscapes/test-slope.asc'}

In [ ]:
m = Map(center = (geom.centroid.y, geom.centroid.x))
m.add_layer(WKTLayer(wkt_string = geom.wkt))
m

In [ ]:
def printSomething(text):
    print('Inside print')
    return text
    
def callbackFunction(value):
    A.append(value)
A = []


pool = Pool(processes=4)
for ix in range(126):
    pool.apply_async(partial(printSomething, ix), callback=callbackFunction)
pool.close()
pool.join()

In [ ]:
A

In [ ]:
count = 0.0
for (runfile, value) in usr.mainapi.runfile_done.items():
    count += value

usr.mainapi.loading_widget.value = 0

In [ ]:
usr.mainapi.update_loading()

In [ ]:
usr.mainapi.runfile_done[runfile] = 1

In [ ]:
usr.mainapi.farsite_lst[0].runfile.tofile()

In [ ]:
dt = {usr.mainapi.farsite_lst[0].runfile: 'test'}
dt[usr.mainapi.farsite_lst[0].runfile]

In [ ]:
usr.mainapi.runfile_lst[0].tofile()

In [ ]:
pd.read_pickle('/home/tcaglar/data/test_table.pkl')

In [ ]:
usr.UI

In [ ]:
gdf = gpd.read_file('/mnt/c/Users/tolk1/Dropbox/SDSC/SDGE/data/Lidar/Documents/LiDAR/Distribution/C11/2017_12_11/GIS-Data/C11_Data.gdb/')

In [10]:
m = Map(
    basemap=basemaps.Esri.WorldTopoMap,
    center=(32.7157, -117.1611),
    zoom=10,
    layout=Layout(height='700px')
)

m.add_control(ScaleControl())
m.add_layer(g)

NameError: name 'g' is not defined

In [ ]:
usr.db.gdfsimulation

In [ ]:
usr.mainapi.farsite_lst[0].runfile.configfile.FARSITE_TIMESTEP

In [ ]:
gdf = gpd.read_file(mainapi.farsite_lst[2].runfile.outpath + '_Perimeters.shp')
minutes = gdf['Elapsed_Mi'].unique()

gdf0 = gdf[gdf['Elapsed_Mi'] == minutes[0]]
polygon_lst = [Polygon(value) for value in gdf0['geometry'].values]
multipoly = MultiPolygon()
for poly in polygon_lst:
    multipoly = multipoly.union(poly.buffer(0))
    
multipoly


In [ ]:
crs = gpd.read_file(mainapi.farsite_lst[0].runfile.outpath + '_Perimeters.shp').crs

In [ ]:
geom = gpd.read_file(mainapi.farsite_lst[0].runfile.outpath + '_Perimeters.shp').loc[0, 'geometry']
mainapi.db.gdfsimulation.loc['db8ac00b671441fbbd5f86e277f1b0d3', 'igniteidx'] = 'test'
gs = gpd.GeoSeries(index=['db8ac00b671441fbbd5f86e277f1b0d3'], data=geom)
mainapi.db.gdfsimulation.loc['db8ac00b671441fbbd5f86e277f1b0d3', 'geometry'] = gs

In [ ]:
gdf = gpd.read_file(mainapi.dftable.iloc[0]['filepath'])
gdf.crs

In [ ]:
gdf['geometry'][0]

In [ ]:
gdftable = gpd.GeoDataFrame(dftable)
gdftable.loc['f23196b034474744bdca7df94b13e0f7', 'geometry'] = gdf['geometry'][0]
gdftable

In [79]:
Exception

Exception

In [ ]:
db = Database(fp)

In [ ]:
fpath = mainapi.dftable.loc['3c01d3099b094139873113e19c9f785a', 'filepath']
# geoms = gpd.read_file(fpath)['geometry']
# for geom in geoms:
#     display(geom)
#     break
    
gdf = gpd.read_file(fpath)
row = gdf.iloc[0]
geom = row['geometry']
gdf

In [ ]:
# Setup the database for reading
dftable = pd.read_pickle(fp.dfpath)

# Collect the tables in dataframe format
# Table 1 - ignition



# Table 2 - barrier
gdfbarrier = gpd.GeoDataFrame(dftable[dftable['filetype'] == 'Barrier'])
for (idx, ignition) in gdfbarrier.iterrows():
    geom = gpd.read_file(ignition['filepath']).loc[0,'geometry']
    gdfbarrier.loc[idx, 'shape'] = geom.to_wkb()

gs = gpd.GeoSeries.from_wkb(gdfignition['shape'])
gdfbarrier['geometry'] = gs
gdfbarrier.drop(columns='shape')

# Table 3 - landscape
# Table 4 - simulation

In [ ]:
gdfignition.loc['714f0cf8d475462a97c651642a3d2525', 'shape']

In [ ]:
# .from_wkb()

gdfignition

### Set the pickle file for maria fire

In [ ]:
# Read measured fire perimeter and convert to EPSG=5070
gdf_maria = gpd.read_file('/home/tcaglar/data/maria-wgs84/')

# ignition geodataframe
gdf_ignition = gdf_maria.to_crs(epsg=5070)
# reindex using the objectid
gdf_ignition.set_index('objectid', inplace=True)

# Remove the objectids 21230 and 21233
gdf_ignition.drop(index=[21230, 21233], inplace=True)

# Calculate datetime and add to gdf
gdf_ignition['datetime'] = pd.to_datetime(gdf_ignition['YYYYMMDD'].astype(str) + ':' + gdf_ignition['24_HHMMSS'], 
               format='%Y%m%d:%H:%M:%S')

In [ ]:
# igniteids = gdf_ignition.index
dftable = pd.DataFrame(columns=['filetype', 'objectid', 'filepath', 'datetime'])
for (objectid, row) in gdf_ignition.iterrows():
    igniteidx = uuid.uuid4().hex
    dftable.loc[igniteidx, 'filepath'] = os.path.join('/home/tcaglar/farsite/inputs/maria_ignite','maria_'+str(objectid)+'.shp')
    dftable.loc[igniteidx, 'filetype'] = 'Ignition'
    dftable.loc[igniteidx, 'objectid'] = objectid
    dftable.loc[igniteidx, 'datetime'] = row['datetime']

lcpidx = uuid.uuid4().hex
dftable.loc[lcpidx, 'filepath'] = '/home/tcaglar/farsite/inputs/landscapes/mariafire.lcp'
dftable.loc[lcpidx, 'filetype'] = 'Landscape'

barrieridx = uuid.uuid4().hex
dftable.loc[barrieridx, 'filepath'] = '/home/tcaglar/farsite/inputs/barriers/NoBarrier/NoBarrier.shp'
dftable.loc[barrieridx, 'filetype'] = 'Barrier'

# dftable.to_pickle('/home/tcaglar/data/test_table.pkl')
dftable = pd.read_pickle('/home/tcaglar/data/test_table.pkl')

In [ ]:
dftable